# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("VacationPy.csv", encoding= "utf-8")

city_data_df.count()
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,hobart,-42.8794,147.3294,47.05,64,20,18.41,AU,1662893036
1,sechura,-5.5569,-80.8222,60.21,91,60,9.91,PE,1662893126
2,byron bay,-28.6500,153.6167,60.94,73,94,7.83,AU,1662892550
3,fort nelson,58.8053,-122.7002,35.19,87,40,0.00,CA,1662893000
4,tautira,-17.7333,-149.1500,77.52,73,5,16.35,PF,1662892880


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

#set locations (lat, long) data for the heatmap
locations = city_data_df.loc[:, ["Lat", "Lng"]]


# Plot Heatmap
fig = gmaps.figure()


heat_layer = gmaps.heatmap_layer(locations, weights=city_data_df["Humidity"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
select_cities = city_data_df[(city_data_df["Cloudiness"]<20) & (city_data_df["Humidity"]<30) &
                             (city_data_df["Max Temp"]>70) & (city_data_df["Max Temp"]<80)]
select_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
226,turkmenabat,39.0733,63.5786,75.18,17,0,20.71,TM,1662893345
393,riacho de santana,-13.6092,-42.9389,79.54,28,4,1.90,BR,1662893442
450,soavinandriana,-19.1667,46.7333,79.16,15,1,6.55,MG,1662893275
478,tafresh,34.6920,50.0130,77.02,28,0,7.36,IR,1662893487
533,castra-nova,44.1167,24.0167,79.34,28,9,16.67,RO,1662893525


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = select_cities.loc[:, ["City", "Lat", "Lng", "Country", "Date"]]

hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
#     "keyword": "hotel",
    "key": g_key
}

for i, row in hotel_df.iterrows():
    try:
        curr_lat = row["Lat"]
        curr_lng = row["Lng"]
        params["location"] = f"{curr_lat},{curr_lng}"

        base_places_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        name_address = requests.get(base_places_url, params)
        
        json_resp = name_address.json()
        
        hotel_name = json_resp["results"][0]["name"]
        
        hotel_df["Hotel Name"][i] = hotel_name
        
    except (KeyError, ValueError, IndexError):
        print("Hotel not found for the city of {}".format(row["City"]))


hotel_df

C:\Users\kaush\AppData\Local\Temp\ipykernel_30108\1714881476.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"][i] = hotel_name


,City,Lat,Lng,Country,Date,Hotel Name
226,turkmenabat,39.0733,63.5786,TM,1662893345,"Гостиница ""Yupek Yoly"" (Шелковый путь)"
393,riacho de santana,-13.6092,-42.9389,BR,1662893442,Casa Nogueira
450,soavinandriana,-19.1667,46.7333,MG,1662893275,Auberge Restaurant Mangarivotra
478,tafresh,34.6920,50.0130,IR,1662893487,هتل یاس سفید
533,castra-nova,44.1167,24.0167,RO,1662893525,Dimitriu marinel


In [6]:
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
hotel_df

,City,Lat,Lng,Country,Date,Hotel Name
226,turkmenabat,39.0733,63.5786,TM,1662893345,"Гостиница ""Yupek Yoly"" (Шелковый путь)"
393,riacho de santana,-13.6092,-42.9389,BR,1662893442,Casa Nogueira
450,soavinandriana,-19.1667,46.7333,MG,1662893275,Auberge Restaurant Mangarivotra
478,tafresh,34.6920,50.0130,IR,1662893487,هتل یاس سفید
533,castra-nova,44.1167,24.0167,RO,1662893525,Dimitriu marinel


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))